In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile1)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile2)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>MMBtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>MMBtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>MMBtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>MMBtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>MMBtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, income, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}_{income}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
     
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
     
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
     
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']])

In [12]:
community = 'north_birmingham'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_tiered.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2784.0
Basic Enclosure                                                     4919.5
Enhanced Enclosure                                                  6923.8
Mininum Efficiency Heat Pump with Existing Heat Backup             13767.5
Mininum Efficiency Heat Pump with Electric Heat Backup             13870.7
Mininum Efficiency Whole Home Electrification                      14705.3
High Efficiency Heat Pump with Electric Heat Backup                24709.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        29055.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30475.4
High Efficie

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Mininum Efficiency Whole Home Electrification                       7450.699007
Electric Clothes Dryer                                               551.443449
Electric Cooking                                                    3699.568356
Induction Cooking                                                  12295.020944
Heat Pump Clothes Dryer                                            10690.615578
Heat Pump Water Heater                                             12368.694344
Mininum Efficiency Heat Pump with Existing Heat Backup             12421.781716
Mininum Efficiency Heat Pump with Electric Heat Backup             12433.212256
Basic Enclosure                                                    12314.594893
Enhanced Enclosure                                                 12340.998249
High Efficiency Whole Home Electrification                         12445.000000
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        12445.000000
Basic Enclosure + High Effi

In [18]:
metric = "saving_cbill.total_usd"
print("Community summary:")
results = get_summary(community_results, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
saving_results['whole_community_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
saving_results['whole_community_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
saving_results['whole_community_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
saving_results['whole_community_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
results

Community summary:


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
High Efficiency Heat Pump with Electric Heat Backup,1155,12433.212256,752.765108,165.765002,329.700135,1048.324478,1479.985052
Enhanced Enclosure + High Efficiency Whole Home Electrification,1158,12445.000000,709.065753,49.056577,253.390977,970.540442,1728.640656
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1158,12445.000000,693.366472,49.049807,239.756446,949.397552,1687.559404
Basic Enclosure + High Efficiency Whole Home Electrification,1158,12445.000000,690.246453,29.510412,240.188923,942.629840,1706.959359
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1158,12445.000000,674.549269,30.399365,236.599136,937.081554,1680.904171
High Efficiency Whole Home Electrification,1158,12445.000000,558.234188,-88.540531,138.257766,854.303992,1450.784894
Enhanced Enclosure,1148,12340.998249,460.651268,47.703211,142.944359,639.227845,1026.134614
Basic Enclosure,1143,12314.594893,398.751045,42.410174,117.696592,564.438690,902.914178
Mininum Efficiency Heat Pump with Electric Heat Backup,1155,12433.212256,334.012650,-17.294793,87.648743,503.134468,809.766173


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single family detached, gas heating, all income  

Segment 2. Occupied, Single family detached, electric heating, all income   


In [19]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_heating_fuel = ['Natural Gas']

seg2_building_type = ['Single-Family Detached']
seg2_heating_fuel = ['Electricity']


community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) &
                      community_results['build_existing_model.heating_fuel'].isin(seg1_heating_fuel),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                      community_results['build_existing_model.heating_fuel'].isin(seg2_heating_fuel),
                     'segment'] = 'segment2'

In [20]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: nan: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,410,4945.494403,484.242989,73.896275,237.575436,640.150121,961.528454
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,410,4945.494403,475.548088,83.800962,237.209309,611.370991,944.870241
Basic Enclosure + High Efficiency Whole Home Electrification,410,4945.494403,469.973978,67.309907,237.239343,634.340334,941.666430
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,410,4945.494403,461.279585,79.486142,235.471498,609.133309,938.707605
High Efficiency Whole Home Electrification,410,4945.494403,401.227983,22.795065,194.735643,554.985558,854.299983
High Efficiency Heat Pump with Electric Heat Backup,407,4933.706659,391.801805,117.300494,197.447925,524.464528,716.074699
Enhanced Enclosure,404,4895.519812,219.145853,31.057752,63.968032,315.039148,462.308780
Basic Enclosure,400,4874.921027,183.253826,25.057879,52.660875,271.378779,408.027748
Mininum Efficiency Heat Pump with Electric Heat Backup,407,4933.706659,145.483939,-35.816041,44.078407,224.261941,416.001927


Summary for segment: segment1: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
High Efficiency Heat Pump with Electric Heat Backup,392,4225.727612,898.223038,244.579751,551.518231,1197.855425,1490.188832
Enhanced Enclosure,391,4202.509328,609.705848,137.912045,299.163127,828.242459,1101.239432
Basic Enclosure,390,4196.704757,522.889760,122.399898,246.689317,714.975082,967.786354
Enhanced Enclosure + High Efficiency Whole Home Electrification,392,4225.727612,361.223048,-204.885362,107.104888,586.330374,807.494078
Mininum Efficiency Heat Pump with Electric Heat Backup,392,4225.727612,351.465714,-118.801518,177.891520,567.527880,795.518389
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,392,4225.727612,348.795446,-215.908973,92.729503,567.383273,798.667056
Basic Enclosure + High Efficiency Whole Home Electrification,392,4225.727612,337.105583,-232.961321,78.403256,545.053323,787.668485
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,392,4225.727612,324.665416,-254.892594,77.414783,530.889182,776.492656
Mininum Efficiency Heat Pump with Existing Heat Backup,392,4225.727612,259.834148,-68.876760,97.986635,454.324533,632.599367


Summary for segment: segment2: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,356,3273.777985,1497.680085,701.242103,924.246727,1856.862419,2501.727022
Basic Enclosure + High Efficiency Whole Home Electrification,356,3273.777985,1478.825711,701.242103,908.599205,1804.705104,2462.872586
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,356,3273.777985,1467.176702,661.399598,891.048464,1815.668038,2462.948657
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,356,3273.777985,1448.345751,657.865077,869.268031,1786.088180,2415.426504
High Efficiency Whole Home Electrification,356,3273.777985,1316.394090,636.468155,839.627027,1602.599587,2152.728290
High Efficiency Heat Pump with Electric Heat Backup,356,3273.777985,1108.996186,430.160330,601.836225,1388.713866,1911.180045
Enhanced Enclosure,353,3242.969109,632.065567,114.341272,257.076004,837.608002,1341.124291
Mininum Efficiency Heat Pump with Electric Heat Backup,356,3273.777985,595.604443,114.100929,228.912488,799.342914,1241.220886
Mininum Efficiency Heat Pump with Existing Heat Backup,356,3273.777985,595.603189,114.100929,228.912488,799.342914,1241.220886


## 2.1 Upgrade Cost Per Dwelling Unit 

In [21]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2891.3
Basic Enclosure                                                     6464.5
Enhanced Enclosure                                                  8852.1
Mininum Efficiency Heat Pump with Existing Heat Backup             16194.6
Mininum Efficiency Heat Pump with Electric Heat Backup             16443.3
Mininum Efficiency Whole Home Electrification                      19425.5
High Efficiency Heat Pump with Electric Heat Backup                26772.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31746.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     33345.8
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       2050.6
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2896.3
Basic Enclosure                                                     5699.3
Enhanced Enclosure                                                  7654.7
Mininum Efficiency Heat Pump with Electric Heat Backup             15591.5
Mininum Efficiency Heat Pump with Existing Heat Backup             15591.5
High Efficiency Heat Pump with Electric Heat Backup                26662.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31360.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32694.8
High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [22]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.08089
Enhanced Enclosure                                                 0.06888
Heat Pump Water Heater                                             0.05067
High Efficiency Heat Pump with Electric Heat Backup                0.03355
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02137
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01604
Heat Pump Clothes Dryer                                            0.01516
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01046
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01023
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00973
Basic Enclosure + High Efficiency Whole Home Electrification       0.00948
Electric Cooking                                                   0.00502
High Efficiency Whole Home Electrification                         0.00462
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.09862
Enhanced Enclosure                                                 0.08257
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04618
Heat Pump Water Heater                                             0.04617
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04487
Basic Enclosure + High Efficiency Whole Home Electrification       0.04208
High Efficiency Heat Pump with Electric Heat Backup                0.04159
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04105
High Efficiency Whole Home Electrification                         0.03954
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03820
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03820
Heat Pump Clothes Dryer                                            0.01530
Induction Cooking                                                  0.00159
Mininum Effi

 ### Annual Energy Savings Per Dollar Invested

In [23]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.00403
Enhanced Enclosure                                                 0.00343
Heat Pump Water Heater                                             0.00284
Mininum Efficiency Whole Home Electrification                      0.00251
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00223
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00218
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00210
Basic Enclosure + High Efficiency Whole Home Electrification       0.00200
High Efficiency Whole Home Electrification                         0.00197
High Efficiency Heat Pump with Electric Heat Backup                0.00196
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00194
Electric Cooking                                                   0.00166
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00132
Induction Co

segment2


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00351
Basic Enclosure                                                    0.00292
Enhanced Enclosure                                                 0.00245
Heat Pump Water Heater                                             0.00185
Electric Cooking                                                   0.00155
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00141
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00137
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00126
High Efficiency Heat Pump with Electric Heat Backup                0.00123
High Efficiency Whole Home Electrification                         0.00123
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00115
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00115
Electric Clo

 ### Annual Emission Savings Per Dollar Invested

In [24]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.26693
Enhanced Enclosure                                                 0.22932
Heat Pump Water Heater                                             0.18778
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14223
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13756
Mininum Efficiency Whole Home Electrification                      0.13729
Basic Enclosure + High Efficiency Whole Home Electrification       0.13055
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12687
High Efficiency Whole Home Electrification                         0.12472
High Efficiency Heat Pump with Electric Heat Backup                0.12231
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11393
Electric Cooking                                                   0.09686
Mininum Efficiency Heat Pump with Existing Heat Backup             0.07509
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.27353
Mininum Efficiency Whole Home Electrification                      0.24477
Enhanced Enclosure                                                 0.23061
Heat Pump Water Heater                                             0.13367
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.12916
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12560
Basic Enclosure + High Efficiency Whole Home Electrification       0.11797
High Efficiency Heat Pump with Electric Heat Backup                0.11717
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11637
Mininum Efficiency Heat Pump with Existing Heat Backup             0.11637
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11519
High Efficiency Whole Home Electrification                         0.11211
Electric Cooking                                                   0.10019
Heat Pump Cl

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [25]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]

segment1


upgrade_name
High Efficiency Heat Pump with Electric Heat Backup                3795645.9
Enhanced Enclosure                                                 2562294.5
Basic Enclosure                                                    2194413.9
Enhanced Enclosure + High Efficiency Whole Home Electrification    1526430.2
Mininum Efficiency Heat Pump with Electric Heat Backup             1485198.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1473914.5
Basic Enclosure + High Efficiency Whole Home Electrification       1424516.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1371947.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1097988.3
High Efficiency Whole Home Electrification                          653371.7
Heat Pump Water Heater                                              619136.8
Heat Pump Clothes Dryer                                             136298.3
Induction Cooking                                              

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4903072.1
Basic Enclosure + High Efficiency Whole Home Electrification       4841347.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4803210.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4741562.4
High Efficiency Whole Home Electrification                         4309582.0
High Efficiency Heat Pump with Electric Heat Backup                3630607.3
Enhanced Enclosure                                                 2049769.1
Mininum Efficiency Heat Pump with Electric Heat Backup             1949876.7
Mininum Efficiency Heat Pump with Existing Heat Backup             1949872.6
Basic Enclosure                                                    1822695.7
Heat Pump Water Heater                                              429595.2
Heat Pump Clothes Dryer                                             106332.8
Induction Cooking                                              

### Annual Total Energy Savings

In [26]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    303890.4
Basic Enclosure + High Efficiency Whole Home Electrification       300939.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     295912.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        292961.1
High Efficiency Whole Home Electrification                         278650.0
High Efficiency Heat Pump with Electric Heat Backup                221488.4
Mininum Efficiency Whole Home Electrification                      206091.2
Mininum Efficiency Heat Pump with Electric Heat Backup             155075.5
Enhanced Enclosure                                                 127596.2
Basic Enclosure                                                    109290.2
Mininum Efficiency Heat Pump with Existing Heat Backup              90556.1
Heat Pump Water Heater                                              34728.1
Heat Pump Clothes Dryer                                              4222.2

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    150703.0
Basic Enclosure + High Efficiency Whole Home Electrification       148922.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     146336.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        144556.0
High Efficiency Whole Home Electrification                         133545.4
High Efficiency Heat Pump with Electric Heat Backup                107275.3
Enhanced Enclosure                                                  60767.5
Mininum Efficiency Heat Pump with Electric Heat Backup              58951.1
Mininum Efficiency Heat Pump with Existing Heat Backup              58951.0
Basic Enclosure                                                     54035.9
Heat Pump Water Heater                                              17221.3
Mininum Efficiency Whole Home Electrification                        6014.9
Heat Pump Clothes Dryer                                              3181.3

### Annual Total Emission Savings

In [27]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    19912155.1
Basic Enclosure + High Efficiency Whole Home Electrification       19608300.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     19383775.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        19079920.6
High Efficiency Whole Home Electrification                         17634382.5
High Efficiency Heat Pump with Electric Heat Backup                13836953.5
Mininum Efficiency Whole Home Electrification                      11269415.2
Enhanced Enclosure                                                  8530978.5
Mininum Efficiency Heat Pump with Electric Heat Backup              7916340.1
Basic Enclosure                                                     7241656.4
Mininum Efficiency Heat Pump with Existing Heat Backup              5138743.1
Heat Pump Water Heater                                              2294283.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    13756931.3
Basic Enclosure + High Efficiency Whole Home Electrification       13573474.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     13443613.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        13260156.5
High Efficiency Whole Home Electrification                         12218838.8
High Efficiency Heat Pump with Electric Heat Backup                10227312.5
Mininum Efficiency Heat Pump with Electric Heat Backup              5939976.6
Mininum Efficiency Heat Pump with Existing Heat Backup              5939968.4
Enhanced Enclosure                                                  5724741.4
Basic Enclosure                                                     5055598.3
Heat Pump Water Heater                                              1243667.8
Mininum Efficiency Whole Home Electrification                        419285.1
Heat Pump Clothes Dryer                            

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [28]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]

segment1


upgrade_name
High Efficiency Heat Pump with Electric Heat Backup                898.2
Enhanced Enclosure                                                 609.7
Basic Enclosure                                                    522.9
Enhanced Enclosure + High Efficiency Whole Home Electrification    361.2
Mininum Efficiency Heat Pump with Electric Heat Backup             351.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     348.8
Basic Enclosure + High Efficiency Whole Home Electrification       337.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        324.7
Mininum Efficiency Heat Pump with Existing Heat Backup             259.8
High Efficiency Whole Home Electrification                         154.6
Heat Pump Water Heater                                             146.5
Heat Pump Clothes Dryer                                             33.5
Induction Cooking                                                    7.1
Electric Cooking                      

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1497.7
Basic Enclosure + High Efficiency Whole Home Electrification       1478.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1467.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1448.3
High Efficiency Whole Home Electrification                         1316.4
High Efficiency Heat Pump with Electric Heat Backup                1109.0
Enhanced Enclosure                                                  632.1
Mininum Efficiency Heat Pump with Electric Heat Backup              595.6
Mininum Efficiency Heat Pump with Existing Heat Backup              595.6
Basic Enclosure                                                     562.0
Heat Pump Water Heater                                              133.7
Heat Pump Clothes Dryer                                              33.8
Induction Cooking                                                     2.7
Electric Cooking         

### Annual Energy Savings Per Dwelling Unit

In [29]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    71.9
Basic Enclosure + High Efficiency Whole Home Electrification       71.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     70.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        69.3
High Efficiency Whole Home Electrification                         65.9
High Efficiency Heat Pump with Electric Heat Backup                52.4
Mininum Efficiency Whole Home Electrification                      48.8
Mininum Efficiency Heat Pump with Electric Heat Backup             36.7
Enhanced Enclosure                                                 30.4
Basic Enclosure                                                    26.0
Mininum Efficiency Heat Pump with Existing Heat Backup             21.4
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    46.0
Basic Enclosure + High Efficiency Whole Home Electrification       45.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     44.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44.2
High Efficiency Whole Home Electrification                         40.8
High Efficiency Heat Pump with Electric Heat Backup                32.8
Enhanced Enclosure                                                 18.7
Mininum Efficiency Heat Pump with Electric Heat Backup             18.0
Mininum Efficiency Heat Pump with Existing Heat Backup             18.0
Basic Enclosure                                                    16.7
Mininum Efficiency Whole Home Electrification                       7.2
Heat Pump Water Heater                                              5.4
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [30]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64.5
Basic Enclosure + High Efficiency Whole Home Electrification       63.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     62.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        62.0
High Efficiency Whole Home Electrification                         59.2
High Efficiency Heat Pump with Electric Heat Backup                45.9
Mininum Efficiency Whole Home Electrification                      43.3
Mininum Efficiency Heat Pump with Electric Heat Backup             31.5
Enhanced Enclosure                                                 26.4
Basic Enclosure                                                    22.7
Mininum Efficiency Heat Pump with Existing Heat Backup             17.6
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.0
Basic Enclosure + High Efficiency Whole Home Electrification       52.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50.6
High Efficiency Whole Home Electrification                         47.1
High Efficiency Heat Pump with Electric Heat Backup                37.0
Enhanced Enclosure                                                 20.5
Mininum Efficiency Heat Pump with Electric Heat Backup             19.4
Mininum Efficiency Heat Pump with Existing Heat Backup             19.4
Basic Enclosure                                                    18.1
Mininum Efficiency Whole Home Electrification                       8.1
Heat Pump Water Heater                                              6.6
Electric Cooking                                                    2.1
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [31]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4712.1
Basic Enclosure + High Efficiency Whole Home Electrification       4640.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4587.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4515.2
High Efficiency Whole Home Electrification                         4173.1
High Efficiency Heat Pump with Electric Heat Backup                3274.5
Mininum Efficiency Whole Home Electrification                      2666.9
Enhanced Enclosure                                                 2030.0
Mininum Efficiency Heat Pump with Electric Heat Backup             1873.4
Basic Enclosure                                                    1725.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1216.1
Heat Pump Water Heater                                              542.9
Electric Cooking                                                     94.7
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4202.2
Basic Enclosure + High Efficiency Whole Home Electrification       4146.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4106.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4050.4
High Efficiency Whole Home Electrification                         3732.3
High Efficiency Heat Pump with Electric Heat Backup                3124.0
Mininum Efficiency Heat Pump with Electric Heat Backup             1814.4
Mininum Efficiency Heat Pump with Existing Heat Backup             1814.4
Enhanced Enclosure                                                 1765.3
Basic Enclosure                                                    1558.9
Mininum Efficiency Whole Home Electrification                       501.9
Heat Pump Water Heater                                              387.1
Electric Cooking                                                     98.0
Heat Pump Clothes Dryer  

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [32]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, income, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [33]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, income, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [34]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]

energy burden (%) before and after upgrade
segment1


/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_42782/2800418672.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_42782/2800418672.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_42782/2800418672.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
High Efficiency Heat Pump with Electric Heat Backup,8.4,5.8
Enhanced Enclosure,8.4,6.6
Basic Enclosure,8.5,6.8
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.4,7.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.4,7.2
Basic Enclosure + High Efficiency Whole Home Electrification,8.4,7.3
Mininum Efficiency Heat Pump with Electric Heat Backup,8.4,7.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.4,7.3
Mininum Efficiency Heat Pump with Existing Heat Backup,8.4,7.6


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.1,4.8
Basic Enclosure + High Efficiency Whole Home Electrification,10.1,4.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,4.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,4.9
High Efficiency Whole Home Electrification,10.1,5.5
High Efficiency Heat Pump with Electric Heat Backup,10.1,6.1
Enhanced Enclosure,10.1,7.8
Basic Enclosure,10.1,7.9
Mininum Efficiency Heat Pump with Electric Heat Backup,10.1,7.9


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single family detached, gas heating, 0-80% AMI  

Segment 2. Occupied, Single family detached, electric heating, 0-80% AMI      

In [35]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_tiered.parquet")

income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_heating_fuel = ['Natural Gas']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Single-Family Detached']
seg2_heating_fuel = ['Electricity']
seg2_median_income = ['0-30%','30-60%','60-80%']


community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                      community_results['build_existing_model.heating_fuel'].isin(seg1_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) &
                      community_results['build_existing_model.heating_fuel'].isin(seg2_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'

In [36]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: nan: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
High Efficiency Heat Pump with Electric Heat Backup,933,9540.214122,702.055468,153.630782,288.035501,957.668037,1380.352245
Enhanced Enclosure + High Efficiency Whole Home Electrification,936,9552.001866,677.837733,48.793972,253.308470,891.687932,1545.701195
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,936,9552.001866,662.540826,45.499197,239.832130,865.493040,1528.297899
Basic Enclosure + High Efficiency Whole Home Electrification,936,9552.001866,658.826797,29.538960,238.401743,889.006754,1518.386727
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,936,9552.001866,643.531464,34.317677,237.314479,851.005522,1492.467326
High Efficiency Whole Home Electrification,936,9552.001866,540.998953,-77.295273,153.540073,809.379190,1327.427065
Enhanced Enclosure,926,9448.000114,414.603334,40.446996,113.725675,573.417270,960.501334
Basic Enclosure,921,9421.596759,354.828588,36.082770,87.620367,488.501564,848.974398
Mininum Efficiency Heat Pump with Electric Heat Backup,933,9540.214122,309.161686,-19.176947,70.036319,477.476430,796.308660


Summary for segment: segment1: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
High Efficiency Heat Pump with Electric Heat Backup,134,1699.191387,807.248916,195.501759,510.092204,1061.619888,1466.434737
Enhanced Enclosure,134,1699.191387,564.621797,211.588833,297.021540,733.989391,1069.816531
Basic Enclosure,134,1699.191387,491.307100,158.586311,251.601503,655.290063,971.514837
Enhanced Enclosure + High Efficiency Whole Home Electrification,134,1699.191387,355.343253,-137.856006,132.190494,512.551001,784.191244
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,134,1699.191387,344.216248,-177.169999,132.353186,490.725538,778.610380
Basic Enclosure + High Efficiency Whole Home Electrification,134,1699.191387,336.841415,-169.422443,122.361106,510.623845,783.495725
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,134,1699.191387,325.708304,-210.998644,111.118352,493.640667,775.783981
Mininum Efficiency Heat Pump with Electric Heat Backup,134,1699.191387,311.472995,-121.372357,174.043984,473.271923,740.175186
Mininum Efficiency Heat Pump with Existing Heat Backup,134,1699.191387,206.858255,-180.800942,91.382722,377.269444,566.874042


Summary for segment: segment2: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,88,1193.806748,1462.397334,705.102075,985.612652,1857.755448,2117.713707
Basic Enclosure + High Efficiency Whole Home Electrification,88,1193.806748,1444.659517,705.102075,943.649063,1856.652661,2117.713707
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,88,1193.806748,1436.970645,674.842695,953.965452,1840.758701,2091.746264
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,88,1193.806748,1419.250782,674.718597,942.410037,1834.712031,2091.746264
High Efficiency Whole Home Electrification,88,1193.806748,1258.559878,620.229497,820.097740,1597.707293,1829.484229
High Efficiency Heat Pump with Electric Heat Backup,88,1193.806748,1080.458358,460.650399,596.837461,1460.831644,1681.195085
Enhanced Enclosure,88,1193.806748,677.097569,190.840288,324.195245,923.424226,1247.765042
Basic Enclosure,88,1193.806748,613.651181,162.340817,279.901342,880.214951,1073.038964
Mininum Efficiency Heat Pump with Electric Heat Backup,88,1193.806748,564.688765,88.118742,264.587290,855.238518,1127.170744


## 3.1 Upgrade Cost Per Dwelling Unit 

In [37]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2786.2
Basic Enclosure                                                     5946.2
Enhanced Enclosure                                                  8143.3
Mininum Efficiency Heat Pump with Existing Heat Backup             14267.0
Mininum Efficiency Heat Pump with Electric Heat Backup             14488.1
Mininum Efficiency Whole Home Electrification                      17387.3
High Efficiency Heat Pump with Electric Heat Backup                25586.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30315.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     31749.5
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       1858.5
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2745.7
Basic Enclosure                                                     5885.8
Enhanced Enclosure                                                  7798.9
Mininum Efficiency Heat Pump with Electric Heat Backup             14844.7
Mininum Efficiency Heat Pump with Existing Heat Backup             14844.7
High Efficiency Heat Pump with Electric Heat Backup                25993.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30603.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     31935.0
High Efficie

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [38]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.08263
Enhanced Enclosure                                                 0.06934
Heat Pump Water Heater                                             0.04950
High Efficiency Heat Pump with Electric Heat Backup                0.03155
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02150
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01450
Heat Pump Clothes Dryer                                            0.01435
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01084
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01074
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01000
Basic Enclosure + High Efficiency Whole Home Electrification       0.00988
Electric Cooking                                                   0.00541
High Efficiency Whole Home Electrification                         0.00507
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.10426
Enhanced Enclosure                                                 0.08682
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04638
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04500
Heat Pump Water Heater                                             0.04328
Basic Enclosure + High Efficiency Whole Home Electrification       0.04200
High Efficiency Heat Pump with Electric Heat Backup                0.04157
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04093
High Efficiency Whole Home Electrification                         0.03875
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03804
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03804
Heat Pump Clothes Dryer                                            0.01305
Induction Cooking                                                  0.00124
Mininum Effi

 ### Annual Energy Savings Per Dollar Invested

In [39]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.00403
Enhanced Enclosure                                                 0.00338
Heat Pump Water Heater                                             0.00296
Mininum Efficiency Whole Home Electrification                      0.00243
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00212
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00202
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00194
Basic Enclosure + High Efficiency Whole Home Electrification       0.00185
High Efficiency Whole Home Electrification                         0.00180
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00179
High Efficiency Heat Pump with Electric Heat Backup                0.00175
Electric Cooking                                                   0.00172
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00117
Induction Co

segment2


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00313
Basic Enclosure                                                    0.00309
Enhanced Enclosure                                                 0.00257
Heat Pump Water Heater                                             0.00170
Electric Cooking                                                   0.00145
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00141
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00137
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00125
High Efficiency Heat Pump with Electric Heat Backup                0.00123
High Efficiency Whole Home Electrification                         0.00120
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00115
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00115
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [40]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.26564
Enhanced Enclosure                                                 0.22510
Heat Pump Water Heater                                             0.19703
Mininum Efficiency Whole Home Electrification                      0.13307
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13170
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12751
Basic Enclosure + High Efficiency Whole Home Electrification       0.12062
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11733
High Efficiency Whole Home Electrification                         0.11421
High Efficiency Heat Pump with Electric Heat Backup                0.10955
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10663
Electric Cooking                                                   0.10047
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06614
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.28833
Enhanced Enclosure                                                 0.24162
Mininum Efficiency Whole Home Electrification                      0.22435
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.12934
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12561
Heat Pump Water Heater                                             0.12374
Basic Enclosure + High Efficiency Whole Home Electrification       0.11761
High Efficiency Heat Pump with Electric Heat Backup                0.11678
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11552
Mininum Efficiency Heat Pump with Existing Heat Backup             0.11552
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11471
High Efficiency Whole Home Electrification                         0.10988
Electric Cooking                                                   0.09272
Heat Pump Cl

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [41]:
saving_results.columns.values

array(['whole_community_upgrade_cost_mean_usd',
       'whole_community_saving_bill_total_usd',
       'whole_community_saving_emission_total_kgCO2e',
       'whole_community_saving_energy_total_mmbtu',
       'whole_community_saving_electricity_total_kwh',
       'whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu',
       'whole_community_saving_bill_mean_usd',
       'whole_community_saving_emission_mean_kgCO2e',
       'whole_community_saving_energy_mean_mmbtu',
       'whole_community_saving_bill_total_usd_wt_p0.1',
       'whole_community_saving_bill_total_usd_wt_p0.25',
       'whole_community_saving_bill_total_usd_wt_p0.75',
       'whole_community_saving_bill_total_usd_wt_p0.9',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.1',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.25',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.75',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.9',
       'segment1_all_incomes_saving_bill_total_usd_wt_p0.1',
   

In [42]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]

segment1


upgrade_name
High Efficiency Heat Pump with Electric Heat Backup                1371670.4
Enhanced Enclosure                                                  959400.5
Basic Enclosure                                                     834824.8
Enhanced Enclosure + High Efficiency Whole Home Electrification     603796.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      584889.3
Basic Enclosure + High Efficiency Whole Home Electrification        572358.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         553440.7
Mininum Efficiency Heat Pump with Electric Heat Backup              529252.2
Mininum Efficiency Heat Pump with Existing Heat Backup              351491.8
High Efficiency Whole Home Electrification                          277124.2
Heat Pump Water Heater                                              234362.3
Heat Pump Clothes Dryer                                              51668.1
Induction Cooking                                              

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1745819.8
Basic Enclosure + High Efficiency Whole Home Electrification       1724644.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1715465.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1694311.2
High Efficiency Whole Home Electrification                         1502477.3
High Efficiency Heat Pump with Electric Heat Backup                1289858.5
Enhanced Enclosure                                                  808323.6
Basic Enclosure                                                     732580.9
Mininum Efficiency Heat Pump with Electric Heat Backup              674129.3
Mininum Efficiency Heat Pump with Existing Heat Backup              674126.4
Heat Pump Water Heater                                              139382.8
Heat Pump Clothes Dryer                                              32857.4
Induction Cooking                                              

### Annual Total Energy Savings

In [43]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    107840.3
Basic Enclosure + High Efficiency Whole Home Electrification       106931.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     104783.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        103875.1
High Efficiency Whole Home Electrification                          98425.2
High Efficiency Heat Pump with Electric Heat Backup                 76297.2
Mininum Efficiency Whole Home Electrification                       71933.2
Mininum Efficiency Heat Pump with Electric Heat Backup              52134.4
Enhanced Enclosure                                                  46796.6
Basic Enclosure                                                     40688.1
Mininum Efficiency Heat Pump with Existing Heat Backup              28469.3
Heat Pump Water Heater                                              14001.0
Heat Pump Clothes Dryer                                              1601.2

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53502.8
Basic Enclosure + High Efficiency Whole Home Electrification       52895.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     52062.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51454.5
High Efficiency Whole Home Electrification                         46478.1
High Efficiency Heat Pump with Electric Heat Backup                38125.0
Enhanced Enclosure                                                 23952.7
Basic Enclosure                                                    21711.3
Mininum Efficiency Heat Pump with Electric Heat Backup             20404.0
Mininum Efficiency Heat Pump with Existing Heat Backup             20403.9
Heat Pump Water Heater                                              5471.7
Mininum Efficiency Whole Home Electrification                       2000.4
Heat Pump Clothes Dryer                                              996.9
Induction Co

### Annual Total Emission Savings

In [44]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    7081392.8
Basic Enclosure + High Efficiency Whole Home Electrification       6986142.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6879139.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6783889.2
High Efficiency Whole Home Electrification                         6237861.5
High Efficiency Heat Pump with Electric Heat Backup                4762953.5
Mininum Efficiency Whole Home Electrification                      3931561.7
Enhanced Enclosure                                                 3114701.1
Basic Enclosure                                                    2683993.3
Mininum Efficiency Heat Pump with Electric Heat Backup             2625139.1
Mininum Efficiency Heat Pump with Existing Heat Backup             1603328.1
Heat Pump Water Heater                                              932770.4
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4892406.5
Basic Enclosure + High Efficiency Whole Home Electrification       4829079.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4788849.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4725522.7
High Efficiency Whole Home Electrification                         4260853.7
High Efficiency Heat Pump with Electric Heat Backup                3623736.3
Enhanced Enclosure                                                 2249525.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2047279.8
Mininum Efficiency Heat Pump with Existing Heat Backup             2047274.2
Basic Enclosure                                                    2025927.6
Heat Pump Water Heater                                              398502.5
Mininum Efficiency Whole Home Electrification                       143441.3
Heat Pump Clothes Dryer                                        

## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [45]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]

segment1


upgrade_name
High Efficiency Heat Pump with Electric Heat Backup                807.2
Enhanced Enclosure                                                 564.6
Basic Enclosure                                                    491.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    355.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     344.2
Basic Enclosure + High Efficiency Whole Home Electrification       336.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        325.7
Mininum Efficiency Heat Pump with Electric Heat Backup             311.5
Mininum Efficiency Heat Pump with Existing Heat Backup             206.9
High Efficiency Whole Home Electrification                         163.1
Heat Pump Water Heater                                             137.9
Heat Pump Clothes Dryer                                             31.7
Induction Cooking                                                    7.0
Electric Cooking                      

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1462.4
Basic Enclosure + High Efficiency Whole Home Electrification       1444.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1437.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1419.3
High Efficiency Whole Home Electrification                         1258.6
High Efficiency Heat Pump with Electric Heat Backup                1080.5
Enhanced Enclosure                                                  677.1
Basic Enclosure                                                     613.7
Mininum Efficiency Heat Pump with Electric Heat Backup              564.7
Mininum Efficiency Heat Pump with Existing Heat Backup              564.7
Heat Pump Water Heater                                              118.8
Heat Pump Clothes Dryer                                              28.8
Induction Cooking                                                     2.1
Electric Cooking         

### Annual Energy Savings Per Dwelling Unit

In [46]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    63.5
Basic Enclosure + High Efficiency Whole Home Electrification       62.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     61.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        61.1
High Efficiency Whole Home Electrification                         57.9
High Efficiency Heat Pump with Electric Heat Backup                44.9
Mininum Efficiency Whole Home Electrification                      42.3
Mininum Efficiency Heat Pump with Electric Heat Backup             30.7
Enhanced Enclosure                                                 27.5
Basic Enclosure                                                    23.9
Mininum Efficiency Heat Pump with Existing Heat Backup             16.8
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    44.8
Basic Enclosure + High Efficiency Whole Home Electrification       44.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     43.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        43.1
High Efficiency Whole Home Electrification                         38.9
High Efficiency Heat Pump with Electric Heat Backup                31.9
Enhanced Enclosure                                                 20.1
Basic Enclosure                                                    18.2
Mininum Efficiency Heat Pump with Electric Heat Backup             17.1
Mininum Efficiency Heat Pump with Existing Heat Backup             17.1
Mininum Efficiency Whole Home Electrification                       5.8
Heat Pump Water Heater                                              4.7
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [47]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    63.5
Basic Enclosure + High Efficiency Whole Home Electrification       63.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     61.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        61.0
High Efficiency Whole Home Electrification                         58.0
High Efficiency Heat Pump with Electric Heat Backup                44.0
Mininum Efficiency Whole Home Electrification                      41.8
Mininum Efficiency Heat Pump with Electric Heat Backup             29.5
Enhanced Enclosure                                                 26.8
Basic Enclosure                                                    23.4
Mininum Efficiency Heat Pump with Existing Heat Backup             15.2
Heat Pump Water Heater                                              8.8
Electric Cooking                                                    1.9
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    55.1
Basic Enclosure + High Efficiency Whole Home Electrification       54.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        52.7
High Efficiency Whole Home Electrification                         47.9
High Efficiency Heat Pump with Electric Heat Backup                38.9
Enhanced Enclosure                                                 23.8
Basic Enclosure                                                    21.4
Mininum Efficiency Heat Pump with Electric Heat Backup             20.1
Mininum Efficiency Heat Pump with Existing Heat Backup             20.1
Mininum Efficiency Whole Home Electrification                       7.3
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    2.1
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [48]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4167.5
Basic Enclosure + High Efficiency Whole Home Electrification       4111.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4048.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3992.4
High Efficiency Whole Home Electrification                         3671.1
High Efficiency Heat Pump with Electric Heat Backup                2803.1
Mininum Efficiency Whole Home Electrification                      2313.8
Enhanced Enclosure                                                 1833.0
Basic Enclosure                                                    1579.6
Mininum Efficiency Heat Pump with Electric Heat Backup             1544.9
Mininum Efficiency Heat Pump with Existing Heat Backup              943.6
Heat Pump Water Heater                                              548.9
Electric Cooking                                                     98.2
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4098.2
Basic Enclosure + High Efficiency Whole Home Electrification       4045.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4011.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3958.4
High Efficiency Whole Home Electrification                         3569.1
High Efficiency Heat Pump with Electric Heat Backup                3035.4
Enhanced Enclosure                                                 1884.3
Mininum Efficiency Heat Pump with Electric Heat Backup             1714.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1714.9
Basic Enclosure                                                    1697.0
Mininum Efficiency Whole Home Electrification                       417.0
Heat Pump Water Heater                                              339.8
Electric Cooking                                                     90.7
Heat Pump Clothes Dryer  

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [49]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, income, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [50]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, income, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [51]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]

energy burden (%) before and after upgrade
segment1


/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_42782/2800418672.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_42782/2800418672.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_42782/2800418672.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
High Efficiency Heat Pump with Electric Heat Backup,15.6,10.7
Enhanced Enclosure,15.6,12.2
Basic Enclosure,15.6,12.6
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.6,13.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.6,13.3
Basic Enclosure + High Efficiency Whole Home Electrification,15.6,13.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.6,13.4
Mininum Efficiency Heat Pump with Electric Heat Backup,15.6,13.5
Mininum Efficiency Heat Pump with Existing Heat Backup,15.6,14.2


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,21.4,9.9
Basic Enclosure + High Efficiency Whole Home Electrification,21.4,10.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,21.4,10.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,21.4,10.2
High Efficiency Whole Home Electrification,21.4,11.5
High Efficiency Heat Pump with Electric Heat Backup,21.4,12.7
Enhanced Enclosure,21.4,16.1
Basic Enclosure,21.4,16.5
Mininum Efficiency Heat Pump with Electric Heat Backup,21.4,16.7


/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_42782/505849424.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [52]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [53]:
! jupyter nbconvert --output-dir='./results/north_birmingham/' --to=html --no-input saving_results_reports_for_north_birmingham.ipynb

[NbConvertApp] Converting notebook saving_results_reports_for_north_birmingham.ipynb to html
/Users/nmerket/mambaforge/envs/datatools/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'application/javascript' -%}
[NbConvertApp] Writing 658501 bytes to results/north_birmingham/saving_results_reports_for_north_birmingham.html


In [55]:
saving_results.columns.values

array(['whole_community_upgrade_cost_mean_usd',
       'whole_community_saving_bill_total_usd',
       'whole_community_saving_emission_total_kgCO2e',
       'whole_community_saving_energy_total_mmbtu',
       'whole_community_saving_electricity_total_kwh',
       'whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu',
       'whole_community_saving_bill_mean_usd',
       'whole_community_saving_emission_mean_kgCO2e',
       'whole_community_saving_energy_mean_mmbtu',
       'whole_community_saving_bill_total_usd_wt_p0.1',
       'whole_community_saving_bill_total_usd_wt_p0.25',
       'whole_community_saving_bill_total_usd_wt_p0.75',
       'whole_community_saving_bill_total_usd_wt_p0.9',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.1',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.25',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.75',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.9',
       'segment1_all_incomes_saving_bill_total_usd_wt_p0.1',
   

In [70]:
rows = ['Basic Enclosure', 'Enhanced Enclosure', 'Heat Pump Water Heater', 'Mininum Efficiency Heat Pump with Electric Heat Backup']
saving_results.loc[rows, ['whole_community_saving_bill_total_usd', 'whole_community_saving_emission_total_kgCO2e']].to_clipboard()